In [1]:
# First, we need to install the packages
!pip install gradio folium pandas

import pandas as pd
import folium
from folium.plugins import MarkerCluster
import gradio as gr
import re, unicodedata, html

# Path to your CSV
CSV_PATH = "kvk_data_codam.csv"

# Static place options you requested
PLAATS_OPTIES = ["Utrecht", "Rotterdam", "Amsterdam", "Eindhoven"]

# Approximate Netherlands bounds (southwest [lat,lon], northeast [lat,lon])
NL_BOUNDS = [[50.75, 3.2], [53.7, 7.22]]
NL_CENTER = [52.1, 5.3]




[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
def _squash_ws(text: str) -> str:
    """
    Normalize text for robust searching:
    - Unicode normalize & remove accents
    - Lowercase
    - Collapse multiple whitespace/newlines to a single space
    """
    if not isinstance(text, str):
        text = "" if pd.isna(text) else str(text)
    text = unicodedata.normalize("NFKD", text)
    text = "".join(ch for ch in text if not unicodedata.combining(ch))
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text

def _safe(s):
    """Stringify and HTML-escape to safely show in Folium popups/tooltips."""
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return ""
    return html.escape(str(s))

In [3]:
def load_data(csv_path=CSV_PATH):
    """Load CSV, coerce lat/lon, clip to NL bounds, create normalized columns."""
    # Handle potential BOM/encoding issues
    try:
        df = pd.read_csv(csv_path)
    except UnicodeDecodeError:
        df = pd.read_csv(csv_path, encoding="utf-8-sig")

    # Ensure numeric lat/lon
    df["lat"] = pd.to_numeric(df.get("lat"), errors="coerce")
    df["lon"] = pd.to_numeric(df.get("lon"), errors="coerce")

    # Keep points roughly within Netherlands
    df = df[
        df["lat"].between(50.5, 54.0) &
        df["lon"].between(2.5, 8.0)
    ].copy()

    # Ensure required columns exist (empty if missing)
    for col in ["Plaats_RechtsPersoon", "Actief", "Bedrijfsomschrijving",
                "HandelsnaamOnderneming", "Rechtsvorm"]:
        if col not in df.columns:
            df[col] = ""

    # Normalize Actief to 'ja'/'nee'
    def _norm_actief(x):
        s = _squash_ws(x)
        return "ja" if s in {"true", "1", "ja", "y", "yes"} else "nee"
    df["Actief_norm"] = df["Actief"].map(_norm_actief)

    # Normalized text column for robust literal substring matching (no regex)
    df["Bedrijfsomschrijving_norm"] = df["Bedrijfsomschrijving"].map(_squash_ws)

    return df

DF = load_data()
DF.shape

(100, 65)

In [4]:
# 4.1: Peek at the first rows
DF.head(10)

,Id,KVKNummer,VestigingNummer,LaatstGewijzigdTijdstip_MA,HoofdNevenVestiging,Actief,Indicator_Onderneming,Indicator_Vestiging,ImportIndicator,ExportIndicator,...,Provincie_RechtsPersoon,Regio_RechtsPersoon,KrimpGebied_RechtsPersoon,AnticipeerGebied_RechtsPersoon,IbisPlanNaam_RechtsPersoon,lat,lon,Bedrijfsomschrijving,Actief_norm,Bedrijfsomschrijving_norm
0,1178628,15828107,9867709295,2020-04-28 13:36:07.8710000,Hoofdvestiging,Ja,Ja,Ja,Nee,Nee,...,Noord-Holland,Noordwest,Nee,Nee,zz - n.v.t./onbekend,52.309344,4.884548,,ja,
1,13598565,6451423,3772422261,2022-06-28 13:44:49.0010000,Hoofdvestiging,Nee,Ja,Ja,Nee,Nee,...,Noord-Brabant,Zuid,Nee,Nee,zz - n.v.t./onbekend,51.450731,5.500133,,nee,
2,1973929,51804166,3835681349,2022-05-25 12:16:08.2820000,Hoofdvestiging,Ja,Ja,Ja,Nee,Nee,...,Noord-Brabant,Zuid,Nee,Nee,zz - n.v.t./onbekend,51.450962,5.453899,,ja,
3,11735140,63572655,7265274934,2018-01-09 14:28:02.7950000,Hoofdvestiging,Nee,Ja,Ja,Nee,Nee,...,Zuid-Holland,Zuidwest,Nee,Nee,zz - n.v.t./onbekend,51.920720,4.433812,,nee,
4,13583448,40644122,7089340237,2019-08-12 11:18:54.0690000,Hoofdvestiging,Nee,Ja,Ja,Nee,Nee,...,Noord-Holland,Noordwest,Nee,Nee,zz - n.v.t./onbekend,52.318757,4.919727,,nee,
5,13918135,13370867,4344308000,2020-10-21 13:38:06.7690000,Hoofdvestiging,Nee,Ja,Ja,Nee,Nee,...,Noord-Holland,Noordwest,Nee,Nee,zz - n.v.t./onbekend,52.298221,4.740187,,nee,
6,639755,94606058,2251818909,2020-01-06 10:56:50.8420000,Hoofdvestiging,Ja,Ja,Ja,Nee,Nee,...,Noord-Holland,Noordwest,Nee,Nee,zz - n.v.t./onbekend,52.363716,4.969677,,ja,
7,7737361,37303637,7884514131,2014-05-07 09:12:48.9990000,Hoofdvestiging,Ja,Ja,Ja,Nee,Nee,...,Noord-Holland,Noordwest,Nee,Nee,zz - n.v.t./onbekend,52.358780,4.832645,,ja,
8,678980,42899343,8289353678,2016-04-18 12:17:05.0940000,Hoofdvestiging,Nee,Ja,Ja,Nee,Nee,...,Noord-Brabant,Zuid,Nee,Nee,zz - n.v.t./onbekend,51.446787,5.451773,,nee,
9,9985800,88017093,1843406482,2014-04-12 06:00:23.8290000,Hoofdvestiging,Nee,Ja,Ja,Nee,Nee,...,Zuid-Holland,Zuidwest,Nee,Nee,zz - n.v.t./onbekend,51.884337,4.517200,,nee,


In [5]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 65 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Id                                100 non-null    int64  
 1   KVKNummer                         100 non-null    int64  
 2   VestigingNummer                   100 non-null    int64  
 3   LaatstGewijzigdTijdstip_MA        100 non-null    object 
 4   HoofdNevenVestiging               100 non-null    object 
 5   Actief                            100 non-null    object 
 6   Indicator_Onderneming             100 non-null    object 
 7   Indicator_Vestiging               100 non-null    object 
 8   ImportIndicator                   100 non-null    object 
 9   ExportIndicator                   100 non-null    object 
 10  Indicator_Faillissement           100 non-null    object 
 11  ActiviteitCode                    100 non-null    int64  
 12  AanvangDa

In [6]:
DF.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Id,100.0,NaN,NaN,NaN,7635941.15,4296150.463496,420833.0,4046099.0,6694229.0,11816137.5,14295321.0
KVKNummer,100.0,NaN,NaN,NaN,47577290.32,30259395.113832,1160675.0,20921116.25,46521353.5,76500972.25,99732835.0
VestigingNummer,100.0,NaN,NaN,NaN,5026408725.46,2736167873.729667,82123982.0,2613874723.5,5208640498.5,7369900639.25,9927789931.0
LaatstGewijzigdTijdstip_MA,100,99,2010-05-25 23:59:59.9990000,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HoofdNevenVestiging,100,2,Hoofdvestiging,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
lat,100.0,NaN,NaN,NaN,51.860164,0.350596,51.3818,51.477858,51.899791,51.996724,52.427778
lon,100.0,NaN,NaN,NaN,4.921913,0.430968,4.381954,4.510329,4.846664,5.425354,5.525533
Bedrijfsomschrijving,100,1,,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Actief_norm,100,2,ja,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
DF["Plaats_RechtsPersoon"].value_counts().loc[PLAATS_OPTIES]

Plaats_RechtsPersoon
Utrecht       1
Rotterdam    40
Amsterdam    24
Eindhoven    35
Name: count, dtype: int64

In [8]:
display(DF["Actief"].value_counts(dropna=False))
display(DF["Actief_norm"].value_counts(dropna=False))

Actief
Ja     56
Nee    44
Name: count, dtype: int64

Actief_norm
ja     56
nee    44
Name: count, dtype: int64

In [9]:
def _fit_or_default(m, df):
    """Fit map to filtered points if any, otherwise to NL bounds."""
    if not df.empty:
        coords = df[["lat", "lon"]].dropna().values.tolist()
        if coords:
            m.fit_bounds(coords, padding=(20, 20))
            return
    m.fit_bounds(NL_BOUNDS, padding=(20, 20))

def build_map(plaatsen_selected, actief_selected, omschrijving_query):
    """Create Folium map based on filters, with rich popups."""
    df = DF.copy()

    # Filter: place (multi-select)
    if plaatsen_selected:
        df = df[df["Plaats_RechtsPersoon"].isin(plaatsen_selected)]

    # Filter: active (ja/nee, multi)
    if actief_selected:
        actief_sel_norm = [_squash_ws(s) for s in actief_selected]
        df = df[df["Actief_norm"].isin(actief_sel_norm)]

    # Filter: free-text on normalized description (literal partial match, no regex)
    q = _squash_ws(omschrijving_query or "")
    if q:
        df = df[df["Bedrijfsomschrijving_norm"].str.contains(q, na=False, regex=False)]

    # Build map
    m = folium.Map(
        location=NL_CENTER,
        zoom_start=7,
        tiles="OpenStreetMap",
        max_bounds=True,
        control_scale=True,
        zoom_control=True,
    )

    cluster = MarkerCluster().add_to(m)

    for _, row in df.dropna(subset=["lat", "lon"]).iterrows():
        plaats   = _safe(row.get("Plaats_RechtsPersoon", ""))
        actief   = _safe(row.get("Actief", row.get("Actief_norm", "")))
        naam     = _safe(row.get("HandelsnaamOnderneming", ""))
        rechtsv  = _safe(row.get("Rechtsvorm", ""))
        desc     = _safe(str(row.get("Bedrijfsomschrijving", "")).strip())

        # Tooltip prefers the trade name; falls back to place or generic label
        tooltip_txt = naam if naam else (plaats if plaats else "Location")

        # Rich popup content
        popup_lines = []
        if naam:    popup_lines.append(f"<b>Handelsnaam:</b> {naam}")
        if rechtsv: popup_lines.append(f"<b>Rechtsvorm:</b> {rechtsv}")
        if plaats:  popup_lines.append(f"<b>Plaats:</b> {plaats}")
        if actief:  popup_lines.append(f"<b>Actief:</b> {actief}")
        if desc:    popup_lines.append(f"<b>Omschrijving:</b> {desc}")
        popup_html = "<br>".join(popup_lines) if popup_lines else "No details"

        folium.Marker(
            [row["lat"], row["lon"]],
            tooltip=tooltip_txt,
            popup=folium.Popup(popup_html, max_width=420),
        ).add_to(cluster)

    _fit_or_default(m, df)
    return m._repr_html_()

In [10]:
# Simple CSS: sticky sidebar + tall map area
CSS = """
#sidebar { position: sticky; top: 0; align-self: flex-start; }
#mapwrap { height: 80vh; }
"""

# Initial HTML for the map (all places, both active states, empty text)
init_html = build_map(
    plaatsen_selected=PLAATS_OPTIES,
    actief_selected=["ja", "nee"],
    omschrijving_query=""
)

with gr.Blocks(title="KVK Map — Places / Active / Description", css=CSS) as demo:
    gr.Markdown("## 🗺️ KVK Map — Filters & Details")

    with gr.Row():
        with gr.Column(scale=1, min_width=280, elem_id="sidebar"):
            plaatsen = gr.CheckboxGroup(
                choices=PLAATS_OPTIES,
                value=PLAATS_OPTIES,
                label="Plaats_RechtsPersoon"
            )
            actief = gr.CheckboxGroup(
                choices=["ja", "nee"],
                value=["ja", "nee"],
                label="Actief"
            )
            omschrijving = gr.Textbox(
                label="Bedrijfsomschrijving (partial, multi-line OK)",
                placeholder="e.g., orthopedie, netwerk, sport..."
            )
            gr.Markdown("Click a marker for **HandelsnaamOnderneming**, **Rechtsvorm**, and more.")

        with gr.Column(scale=3, elem_id="mapwrap"):
            out = gr.HTML(value=init_html)

    # Live interactions (textbox updates on input for instant filtering)
    plaatsen.change(build_map, inputs=[plaatsen, actief, omschrijving], outputs=out)
    actief.change(build_map, inputs=[plaatsen, actief, omschrijving], outputs=out)
    omschrijving.input(build_map, inputs=[plaatsen, actief, omschrijving], outputs=out)

# In notebooks, simply run demo.launch(). In a script, use if __name__ == "__main__".
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
